In [ ]:
# Importar las secuencias a QIIME2 demultiplexadas

qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path manifest_bacterias.txt \
  --output-path demux_bacterias.qza \
  --input-format SingleEndFastqManifestPhred33V2


In [ ]:
# Hacer un resumen de lo que encontró de las secuencias
qiime demux summarize \
--i-data demux_bacterias.qza \
--o-visualization SecuenciasBacterias # Acá está toda la info

In [ ]:
# DADA2 
qiime dada2 denoise-single \
--i-demultiplexed-seqs demux_bacterias.qza \
--p-trim-left 17 \
--p-trunc-len 270 \
--o-table TablaDADA2Bacterias \
--o-representative-sequences SeqRepresentativasBacterias \
--o-denoising-stats StatsDenoisingBacterias \
--verbose

In [ ]:
# Tabla de features 
qiime feature-table summarize \
--i-table TablaDADA2Bacterias.qza
--o-visualization TablaDADA2Bacterias \
--m-sample-metadata-file metadatos_bacterias_qiime.txt #Estos son los metadatos

In [ ]:
qiime metadata tabulate \
--m-input-file StatsDenoisingBacterias.qza \
--o-visualization StatsDenoisingBacterias

In [ ]:
#Arboles
qiime phylogeny align-to-tree-mafft-fasttree \
--i-sequences rep-seqs.qza \
--o-alignment SeqRepAlineadasBacterias \
--o-masked-alignment SeqRepMaskedBacterias \
--o-tree ArbolUnrootedBacterias \
--o-rooted-tree ArbolRootedBacterias

In [ ]:
#Rarefaccion bacterias
qiime diversity alpha-rarefaction \
  --i-table TablaDADA2Bacterias.qza \
  --i-phylogeny ArbolRootedBacterias.qza \
  --p-max-depth 86845 \
  --m-metadata-file metadatos_bacterias.txt \
  --o-visualization RarefactionBacterias.qzv 

In [ ]:
#Tabla DADA filtrada para filósfera
qiime feature-table filter-samples \
  --i-table TablaDADA2Bacterias.qza \
  --m-metadata-file metadatos_bacterias_qiime.txt \
  --p-where "Tipo_muestra='Filosfera'" \
  --o-filtered-table TablaFilosfera.qza


In [ ]:
# Tabla DADA filtrada para rizosfera
qiime feature-table filter-samples \
  --i-table TablaDADA2Bacterias.qza \
  --m-metadata-file metadatos_bacterias_qiime.txt \
  --p-where "Tipo_muestra='Rizosfera'" \
  --o-filtered-table TablaRizosfera.qza

In [ ]:
# Rarefacción solo filósfera
nohup qiime diversity alpha-rarefaction   
--i-table TablaFilosfera.qza   
--i-phylogeny ArbolRootedBacterias.qza   
--p-max-depth 14000   
--m-metadata-file metadatos_bacterias_qiime.txt   
--o-visualization RarefactionFilosfera.qzv &

In [ ]:
# Rarefacción solo rizósfera
nohup qiime diversity alpha-rarefaction   
--i-table TablaRizosfera.qza   
--i-phylogeny ArbolRootedBacterias.qza   
--p-max-depth 100000   
--m-metadata-file metadatos_bacterias_qiime.txt   
--o-visualization RarefactionRizosfera.qzv &

In [ ]:
# Diversidad bacterias
qiime diversity core-metrics-phylogenetic \
--i-phylogeny ArbolRootedBacterias.qza \
--i-table TablaDADA2Bacterias.qza \
--p-sampling-depth 4000 \ 
--m-metadata-file metadatos_bacterias_qiime.txt \ 
--output-dir ResultadosDiversidadBacterias

In [ ]:
# Diversidad filosfera
qiime diversity core-metrics-phylogenetic 
--i-phylogeny ArbolRootedBacterias.qza 
--i-table TablaFilosfera.qza 
--p-sampling-depth 4000 
--m-metadata-file metadatos_bacterias_qiime.txt 
--output-dir ResultadosDiversidadFilosfera

In [ ]:
# Entrenar clasificador

qiime feature-classifier fit-classifier-naive-bayes 
--i-reference-reads 2022.10.seqs.fna.qza 
--i-reference-taxonomy 2022.10.taxonomy.asv.tsv.qza 
--o-classifier ClasificadorGG 
--verbose

In [ ]:
# Asignación taxonomica bacterias
qiime feature-classifier classify-sklearn \
--i-classifier Greengenes2-2022/ClasificadorGG.qza \
--i-reads SeqRepresentativasBacteri.qza \
--o-classification TaxonomiaBacterias.qza \
--verbose

In [ ]:
#Taxonomia
qiime metadata tabulate \
--m-input-file TaxonomiaBacterias.qza \
--o-visualization TaxonomiaBacterias.qzv

# Barplots
qiime taxa barplot \
--i-table table.qza \
--i-taxonomy TaxonomiaBacterias.qza \
--m-metadata-file metadatos_bacterias_qiime.txt \
--o-visualization BarplotBacterias.qzv

In [ ]:
# Diferencias en las diversidades alfa (Solo como por mirar, yo las repito en R)
qiime diversity alpha-group-significance \
--i-alpha-diversity ResultadosDiversidadBacterias/shannon_vector.qza \
--m-metadata-file metadatos_bacterias_qiime.txt \
--o-visualization ResultadosDiversidadBacterias/shannon-ANOVA